# Random Forest (sklearn) vs XGBoost

# В этом ноутбуке мы исследуем качество работы композиций алгоритмов на основе ансамблей деревьев в зависимости от числа деревьев в композиции

## sklearn.ensemble.RandomForestClassifier

In [0]:
from sklearn import ensemble , model_selection, metrics 
from sklearn.model_selection import learning_curve

import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgbm

import warnings
warnings.filterwarnings('ignore')

In [0]:
%pylab inline

### Данные

Задача на kaggle: https://www.kaggle.com/gsdeepakkumar/visualisation-with-python-hr-data

По характеристикам работы сотрудника требуется определить, уволится ли он из компании в течение ближайшего года.

Для демонстрации используется обучающая выборка из исходных данных HR.csv, файл с данными прилагается.

In [0]:
!wget https://raw.githubusercontent.com/dvpolyakov/ml_course/master/HR.csv

In [0]:
HR_data = pd.read_csv('HR.csv', header=0, sep=',')

In [0]:
HR_data.head()

- `last_evaluation` - рейтинг сотрудника по итогам последнего ревью (от 0 до 1)
- `number_project` - число успешно завершенных проектов за последний год
- `average_montly_hours` - среднее число отработанных в месяц часов
- `time_spend_company` - сколько лет сотрудник уже работает в компании
- `Work_accident` - случались ли серьезные рабочие конфликты за время работы
- `promotion_last_5years` - сколько раз за повледние 5 лет повышалась должность сотрудника

- `left` - ушел ли сотрудник из компании в течение следующего года (целевая переменная)

In [0]:
HR_target = HR_data['left'].values

HR_data.drop(['left'], axis=1, inplace=True)
HR_data.head()

## Зависимость качества от количества деревьев

### RandomForestClassifier

In [0]:
n_trees = [1] + list(range(5, 55, 5))

In [0]:
ensemble.RandomForestClassifier?

In [0]:
from tqdm import tqdm_notebook 

In [0]:
%%time
scoring = []
for n_tree in tqdm_notebook(n_trees):
    estimator = ensemble.RandomForestClassifier(n_estimators = n_tree, min_samples_split=5, random_state=1)
    score = model_selection.cross_val_score(
        estimator, HR_data, HR_target, 
        scoring = 'accuracy', cv = 3
    )    
    scoring.append(score)
scoring = np.asmatrix(scoring)

In [0]:
plt.figure(figsize=(9, 6))
pylab.plot(n_trees, scoring.mean(axis = 1), marker='.', label='RandomForest')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right');

### XGBoost

In [0]:
%%time
xgb_scoring = []
for n_tree in tqdm_notebook(n_trees):
    estimator = xgb.XGBClassifier(learning_rate=0.1, max_depth=6, n_estimators=n_tree)
    score = model_selection.cross_val_score(
        estimator, HR_data, HR_target,
        scoring = 'accuracy', cv = 3
    )    
    xgb_scoring.append(score)
xgb_scoring = np.asmatrix(xgb_scoring)

## Сравним метрики качества RandomForest и XGBoost

In [0]:
plt.figure(figsize=(9, 6))
pylab.plot(n_trees, scoring.mean(axis = 1), marker='.', label='RandomForest')
pylab.plot(n_trees, xgb_scoring.mean(axis = 1), marker='.', label='XGBoost')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right')

Видим, что градиентный бустинг на этих данных работает лучше

**Задание**

Поэкспериментируйте с параметрами модели XGBoost чтобы добиться лучшего качества.

Вот лишь малый список параметров, которые можно оптимизировать:
- темп обучения (`learning_rate`)
- число деревьев в композиции (`n_estimators`)
- максимальная глубина деревьев (`max_depth`)

## Теперь проинтерпретируем предсказания моделей с помощью библиотеки SHAP values

- [Ссылка на основной репозиторий модуля](https://github.com/slundberg/shap)

- [Статья с примерами применения SHAP values](https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d)

- [Документация по модулю SHAP](https://christophm.github.io/interpretable-ml-book/shap.html)

## SHAP values пришли из кооперативной теории игр. Позволяют получать несмещенные оценки важности признаков для каждого объекта

In [0]:
!pip install shap

In [0]:
import shap
shap.initjs()

### Создайте объект `XGBClassifier` с параметрами 

- `objective="binary:logistic"`
- `learning_rate=0.1`
- `max_depth=6`
- `n_estimators=50`

In [0]:
clf = xgb.XGBClassifier(#YOUR CODE HERE)
clf.fit(HR_data, HR_target, verbose=True)

In [0]:
explainer = shap.TreeExplainer(clf, feature_dependence="independent")
shap_values = explainer.shap_values(HR_data)

## Посмотрим на предсказание для отдельного работника и на факторы, которые по мнению модели являются самыми важными

In [0]:
shap.initjs()
worker_id = 0
shap.force_plot(explainer.expected_value, shap_values[worker_id,:], HR_data.iloc[worker_id,:], link='logit')

## Посмотрим картину для всех объектов вместе

In [0]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, HR_data, link='logit')

## Посмотрим на рапеределение признаков и на их влияение на целевую переменную

In [0]:
shap.summary_plot(shap_values, HR_data)

#### **Материалы по xgboost:**
python api: http://xgboost.readthedocs.org/en/latest/python/python_api.html

### Catboost

In [0]:
!pip install catboost

In [0]:
import catboost

%%time
catboost_scoring = []
for n_tree in tqdm_notebook(n_trees):
    estimator = catboost.CatBoostClassifier(max_depth=6, n_estimators=n_tree, verbose=False)
    score = model_selection.cross_val_score(
        estimator, bioresponce_data, bioresponce_target,
        scoring = 'accuracy', cv=3
    ) 
    catboost_scoring.append(score)
catboost_scoring = np.asmatrix(catboost_scoring)

#### **Материалы по Catboost:**
python api: https://tech.yandex.com/catboost/doc/dg/concepts/python-quickstart-docpage/

### LightGBM

In [0]:
%%time
lgbm_scoring = []
for n_tree in tqdm_notebook(n_trees):
    estimator = lgbm.LGBMClassifier(learning_rate=0.1, max_depth=6, n_estimators=n_tree)
    score = model_selection.cross_val_score(
        estimator, bioresponce_data, bioresponce_target,
        scoring='accuracy', cv=3
    )    
    lgbm_scoring.append(score)
lgbm_scoring = np.asmatrix(lgbm_scoring)

#### **Материалы по LightGMB:**
https://lightgbm.readthedocs.io/en/latest/Python-API.html#scikit-learn-api

In [0]:
plt.figure(figsize=(9, 6))

pylab.plot(n_trees, scoring.mean(axis=1), marker='.', label='RandomForest')
pylab.plot(n_trees, xgb_scoring.mean(axis=1), marker='.', label='XGBoost')
pylab.plot(n_trees, catboost_scoring.mean(axis=1), marker='.', label='Catboost')
pylab.plot(n_trees, lgbm_scoring.mean(axis=1), marker='.', label='LGBMClassifier')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right');